# **📂 E-Commerce Sales Data Analysis**

In [1]:
import os

print(os.listdir("D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Raw"))

['online_retail_II.xlsx', 'Year 2009-2010.csv', 'Year 2010-2011.csv']


### **Load & Combine Raw Data**

In [2]:
## STEP 1 — Load and Combine Raw Files

import pandas as pd

file_1 = "D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Raw\Year 2009-2010.csv"
file_2 = "D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Raw\Year 2010-2011.csv"

df1 = pd.read_csv(file_1, encoding='latin-1')
df2 = pd.read_csv(file_2, encoding='latin-1')

df = pd.concat([df1, df2], ignore_index=True)

print("Raw shape:", df.shape)
df.head()

Raw shape: (1067371, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom


### **Standardize Columns & Core Cleaning**

In [3]:
## STEP 2 — Standardize Columns and Clean Data

df = df.rename(columns={
    'Customer ID': 'CustomerID',
    'Price': 'UnitPrice'
})

# Remove cancelled invoices
df = df[~df['Invoice'].astype(str).str.startswith('C')]

# Remove invalid values
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]

# Convert dates
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df = df[df['InvoiceDate'].notna()]

# Enforce valid Customer IDs
df['CustomerID'] = pd.to_numeric(df['CustomerID'], errors='coerce')
df = df[df['CustomerID'].notna()]

# Business fields
df['Revenue'] = df['Quantity'] * df['UnitPrice']
df['InvoiceDateOnly'] = df['InvoiceDate'].dt.date
df['InvoiceYear'] = df['InvoiceDate'].dt.year
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')

df = df.drop_duplicates()

print("Cleaned shape:", df.shape)
df.head()

Cleaned shape: (779425, 12)


,Invoice,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,InvoiceDateOnly,InvoiceYear,InvoiceMonth
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009-12-01,2009,2009-12
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12-01,2009,2009-12
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12-01,2009,2009-12
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,2009-12-01,2009,2009-12
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,2009-12-01,2009,2009-12


### **Build ORDERS Table**

In [4]:
## STEP 3 — Build Orders Table (Invoice-level)

orders = df.groupby(
    ['Invoice','CustomerID','Country','InvoiceDateOnly']
).agg({
    'Revenue':'sum',
    'Quantity':'sum'
}).reset_index()

print("Orders shape:", orders.shape)
orders.head()

Orders shape: (36969, 6)


,Invoice,CustomerID,Country,InvoiceDateOnly,Revenue,Quantity
0,489434,13085.0,United Kingdom,2009-12-01,505.30,166
1,489435,13085.0,United Kingdom,2009-12-01,145.80,60
2,489436,13078.0,United Kingdom,2009-12-01,630.33,193
3,489437,15362.0,United Kingdom,2009-12-01,310.75,145
4,489438,18102.0,United Kingdom,2009-12-01,2286.24,826


### **Build CUSTOMERS Table + RFM & Segments**

In [5]:
## STEP 4 — Build Customers Table + RFM Segmentation

customers = df.groupby('CustomerID').agg({
    'InvoiceDate':'max',
    'Revenue':'sum',
    'Invoice':'nunique'
}).reset_index()

customers = customers.rename(columns={
    'InvoiceDate':'LastPurchaseDate',
    'Revenue':'TotalRevenue',
    'Invoice':'TotalOrders'
})

snapshot_date = df['InvoiceDate'].max()

customers['RecencyDays'] = (snapshot_date - customers['LastPurchaseDate']).dt.days
customers['Frequency'] = customers['TotalOrders']
customers['Monetary'] = customers['TotalRevenue']

# RFM scoring
customers['R_Score'] = pd.qcut(customers['RecencyDays'], 5, labels=[5,4,3,2,1]).astype(int)
customers['F_Score'] = pd.qcut(customers['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
customers['M_Score'] = pd.qcut(customers['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

customers['RFM_Score'] = (
    customers['R_Score'].astype(str) +
    customers['F_Score'].astype(str) +
    customers['M_Score'].astype(str)
)

def segment_customer(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
    if r >= 4 and f >= 4 and m >= 4:
        return "Champions"
    if r >= 3 and f >= 3:
        return "Loyal Customers"
    if r >= 4 and f <= 2:
        return "New Customers"
    if r == 3 and f <= 2:
        return "Promising"
    if r <= 2 and f >= 3:
        return "At Risk"
    if r == 1 and f >= 2:
        return "Need Attention"
    return "Low Value"

customers['Segment'] = customers.apply(segment_customer, axis=1)

print("Customers shape:", customers.shape)
customers.head()

Customers shape: (5878, 12)


,CustomerID,LastPurchaseDate,TotalRevenue,TotalOrders,RecencyDays,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,12346.0,2011-01-18 10:01:00,77556.46,12,325,12,77556.46,2,5,5,255,At Risk
1,12347.0,2011-12-07 15:52:00,4921.53,8,1,8,4921.53,5,4,5,545,Champions
2,12348.0,2011-09-25 13:13:00,2019.40,5,74,5,2019.40,3,4,4,344,Loyal Customers
3,12349.0,2011-11-21 09:51:00,4428.69,4,18,4,4428.69,5,3,5,535,Loyal Customers
4,12350.0,2011-02-02 16:01:00,334.40,1,309,1,334.40,2,1,2,212,Low Value


### **Build PRODUCT SALES Table**

In [6]:
## STEP 5 — Build Product Sales Table

product_sales = df.groupby(['StockCode','Description']).agg({
    'Revenue':'sum',
    'Quantity':'sum',
    'Invoice':'nunique'
}).reset_index().rename(columns={'Invoice':'OrderCount'})

print("Product table shape:", product_sales.shape)
product_sales.head()

Product table shape: (5315, 5)


,StockCode,Description,Revenue,Quantity,OrderCount
0,10002,INFLATABLE POLITICAL GLOBE,6638.27,8479,297
1,10080,GROOVY CACTUS INFLATABLE,124.61,303,26
2,10109,BENDY COLOUR PENCILS,1.68,4,1
3,10120,DOGGY RUBBER,136.08,648,62
4,10123C,HEARTS WRAPPING TAPE,226.76,628,46


### **Build Monthly KPI Table**

In [7]:
## STEP 6 — Build Monthly KPI Table

monthly_kpi = df.groupby(df['InvoiceDate'].dt.to_period('M')).agg(
    Revenue=('Revenue','sum'),
    Orders=('Invoice','nunique'),
    Customers=('CustomerID','nunique')
).reset_index()

monthly_kpi['AOV'] = monthly_kpi['Revenue'] / monthly_kpi['Orders']

print("Monthly KPI shape:", monthly_kpi.shape)
monthly_kpi.head()

Monthly KPI shape: (25, 5)


,InvoiceDate,Revenue,Orders,Customers,AOV
0,2009-12,683504.010,1512,955,452.052917
1,2010-01,555802.672,1011,720,549.755363
2,2010-02,504558.956,1104,772,457.028040
3,2010-03,696978.471,1524,1057,457.334955
4,2010-04,591982.002,1329,942,445.434163


### **Build Cohort Retention Table**

In [12]:
## FINAL — Cohort Table for Power BI (Stable & Independent)

cohort_df = df[['CustomerID','InvoiceDate']].copy()

cohort_df['OrderMonth'] = cohort_df['InvoiceDate'].dt.to_period('M')
cohort_df['CohortMonth'] = cohort_df.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')

cohort_df['CohortPeriod'] = (
    (cohort_df['OrderMonth'].dt.to_timestamp() -
     cohort_df['CohortMonth'].dt.to_timestamp()).dt.days // 30
)

cohort_retention = (
    cohort_df.groupby(['CohortMonth','CohortPeriod'])['CustomerID']
    .nunique()
    .reset_index()
)

# convert to retention percent by cohort
cohort_sizes = cohort_retention[cohort_retention['CohortPeriod'] == 0][['CohortMonth','CustomerID']]
cohort_sizes = cohort_sizes.rename(columns={'CustomerID':'CohortSize'})

cohort_retention = cohort_retention.merge(cohort_sizes, on='CohortMonth', how='left')
cohort_retention['Retention'] = (cohort_retention['CustomerID'] / cohort_retention['CohortSize']).round(3)

cohort_retention = cohort_retention[['CohortMonth','CohortPeriod','Retention']]

## Fill missing CohortPeriods with 0% retention (true zero months)

all_periods = (
    cohort_retention
    .groupby('CohortMonth')['CohortPeriod']
    .max()
    .max()
)

index = pd.MultiIndex.from_product(
    [cohort_retention['CohortMonth'].unique(),
     range(int(all_periods)+1)],
    names=['CohortMonth','CohortPeriod']
)

cohort_retention = (
    cohort_retention
    .set_index(['CohortMonth','CohortPeriod'])
    .reindex(index, fill_value=0)
    .reset_index()
)

Cohort table exported — no other tables changed.


In [23]:
bad_groups = (
    cohort_base
      .groupby(['CohortMonth','CustomerID'])['CohortPeriod']
      .nunique()
      .reset_index(name='period_count')
      .query('period_count > 1')
)

print("Customers with multiple periods in same row-group:", len(bad_groups))
print(bad_groups.head(20))


Customers with multiple periods in same row-group: 4087
   CohortMonth  CustomerID  period_count
0      2009-12     12346.0             5
1      2009-12     12358.0             5
2      2009-12     12359.0             8
3      2009-12     12362.0             9
4      2009-12     12417.0            16
5      2009-12     12422.0            11
6      2009-12     12435.0             6
7      2009-12     12437.0            20
8      2009-12     12439.0             2
9      2009-12     12440.0             2
11     2009-12     12455.0             9
12     2009-12     12471.0            23
13     2009-12     12472.0            16
15     2009-12     12490.0            17
16     2009-12     12510.0             7
17     2009-12     12523.0            18
18     2009-12     12533.0             2
19     2009-12     12539.0             5
20     2009-12     12540.0            22
21     2009-12     12557.0             8


In [24]:
sample = cohort_base.merge(bad_groups, on=['CohortMonth','CustomerID'], how='inner')
print(sample.sort_values(['CustomerID','CohortPeriod']).head(30))

       CustomerID         InvoiceDate OrderMonth CohortMonth  CohortPeriod  \
686       12346.0 2009-12-14 08:34:00    2009-12     2009-12             0   
866       12346.0 2010-01-04 09:24:00    2010-01     2009-12             1   
2297      12346.0 2010-03-02 13:08:00    2010-03     2009-12             3   
5758      12346.0 2010-06-28 13:53:00    2010-06     2009-12             6   
12708     12346.0 2011-01-18 10:01:00    2011-01     2009-12            13   
9978      12347.0 2010-10-31 14:20:00    2010-10     2010-10             0   
11839     12347.0 2010-12-07 14:57:00    2010-12     2010-10             2   
12899     12347.0 2011-01-26 14:30:00    2011-01     2010-10             3   
14782     12347.0 2011-04-07 10:43:00    2011-04     2010-10             6   
16774     12347.0 2011-06-09 13:01:00    2011-06     2010-10             8   
18306     12347.0 2011-08-02 08:48:00    2011-08     2010-10            10   
21544     12347.0 2011-10-31 12:25:00    2011-10     2010-10    

In [21]:
dupes = (
    cohort_base
      .groupby(['CustomerID','CohortMonth','CohortPeriod'])
      .size()
      .reset_index(name='count')
      .query('count > 1')
)

print("Duplicate groups:", len(dupes))
print(dupes.head(20))

Duplicate groups: 0
Empty DataFrame
Columns: [CustomerID, CohortMonth, CohortPeriod, count]
Index: []


In [18]:
## CLEAN COHORT BASE (ONE ROW PER CUSTOMER PER ORDER MONTH)

cohort_base = (
    df[['CustomerID','InvoiceDate']]
      .copy()
)

cohort_base['OrderMonth'] = cohort_base['InvoiceDate'].dt.to_period('M')
cohort_base['CohortMonth'] = (
    cohort_base.groupby('CustomerID')['InvoiceDate']
    .transform('min')
    .dt.to_period('M')
)

# convert to integer period offsets
cohort_base['CohortPeriod'] = (
    (cohort_base['OrderMonth'].dt.to_timestamp() -
     cohort_base['CohortMonth'].dt.to_timestamp())
    .dt.days // 30
)

# remove duplicate purchases in same month
cohort_base = cohort_base.drop_duplicates(
    subset=['CustomerID','CohortMonth','CohortPeriod']
)

print("Rows:", cohort_base.shape)
print(cohort_base.head())

Rows: (25441, 5)
    CustomerID         InvoiceDate OrderMonth CohortMonth  CohortPeriod
0      13085.0 2009-12-01 07:45:00    2009-12     2009-12             0
12     13078.0 2009-12-01 09:06:00    2009-12     2009-12             0
31     15362.0 2009-12-01 09:08:00    2009-12     2009-12             0
54     18102.0 2009-12-01 09:24:00    2009-12     2009-12             0
71     12682.0 2009-12-01 09:28:00    2009-12     2009-12             0


In [25]:
## SURVIVAL RETENTION — NO PIVOT, NO DUPES, NO SHAPE ERRORS

base = cohort_base[['CustomerID','CohortMonth','CohortPeriod']].copy()

# each customer is "active up to" all subsequent periods
base['ActiveUntil'] = base.groupby(['CohortMonth','CustomerID'])['CohortPeriod']\
                          .transform(lambda s: s.cummax())

# mark rows as active
base['Active'] = 1

# cohort size (period 0)
cohort_sizes = (
    base[base['CohortPeriod']==0]
      .groupby('CohortMonth')['CustomerID']
      .nunique()
      .rename('CohortSize')
      .reset_index()
)

# customers still active up to each period
survival = (
    base.groupby(['CohortMonth','CohortPeriod'])['CustomerID']
        .nunique()
        .reset_index(name='ActiveCustomers')
        .merge(cohort_sizes, on='CohortMonth')
)

survival['SurvivalRetention'] = (
    survival['ActiveCustomers'] / survival['CohortSize']
).round(3)

survival = survival[['CohortMonth','CohortPeriod','SurvivalRetention']]

Exported SURVIVAL retention (pivot-free, correct).


In [28]:
## COHORT REVENUE + LTV TABLE (USING EXISTING COLUMNS)

ltv = df[['CustomerID','Revenue','OrderMonth','CohortMonth']].copy()

# ensure period fields exist
ltv['OrderMonth']  = ltv['OrderMonth'].astype('period[M]')
ltv['CohortMonth'] = ltv['CohortMonth'].astype('period[M]')

ltv['CohortPeriod'] = (
    (ltv['OrderMonth'].dt.to_timestamp() -
     ltv['CohortMonth'].dt.to_timestamp())
    .dt.days // 30
)

# aggregate revenue by cohort + period
cohort_revenue = (
    ltv.groupby(['CohortMonth','CohortPeriod'])['Revenue']
       .sum()
       .reset_index()
)

# cumulative LTV curve
cohort_revenue['CumulativeRevenue'] = (
    cohort_revenue
        .groupby('CohortMonth')['Revenue']
        .cumsum()
)

Exported cohort revenue + LTV table.


### **Export All Final Tables**

In [9]:
## STEP 8 — Export Final Data Tables

orders.to_csv("D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Clean\orders.csv", index=False)
customers.to_csv("D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Clean\customers.csv", index=False)
product_sales.to_csv("D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Clean\product_sales.csv", index=False)
monthly_kpi.to_csv("D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Clean\monthly_kpi.csv", index=False)
cohort_retention.to_csv("D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Clean\cohort_retention.csv")
survival.to_csv("D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Clean\cohort_retention_survival.csv", index=False)
cohort_revenue.to_csv("D:\Downloads\Documents & Files\Others\ecommerce_sales_project\Data Clean\cohort_revenue_ltv.csv", index=False)

print("Export completed successfully.")

Export completed successfully.
